# Storytelling Twitter with POSLDA

**Author: Yasir Abdurrahman**

# Background Information
**Twitter** merupakan salah satu sosial media yang populer digunakan masyarakat Indonesia. Terbatasnya jumlah karakter *tweet* sebanyak 144 karakter, tidak menjadikan nilai informasi sebuah *tweet* berkurang. Hal ini dibuktikan dengan masuknya *trending topic* tagar **#PilkadaDKI** saat pemilihan gubernur bulan April 2017 kemarin seperti pada berita [CNN](https://www.cnnindonesia.com/teknologi/20170419143713-192-208647/tagar-quick-count-dan-pilkada-dki-kuasai-twitter/). Menilik ke belakang tahun 2014, saat Pemilihan Umum Legislatif (Pemilu) taggar **#IndonesiaElectionDay** juga masuk sebagai *trending topic* pada berita [Liputan6](http://tekno.liputan6.com/read/2034443/hashtag-bertema-pemilu-dominasi-trending-topic-twitter). 

Sebuah informasi sangatlah bernilai bagi jurnalis dalam membuat sebuah berita. Pengumpulan informasi dari Twitter masih terbatas berdasarkan *trending topic*, padahal masih banyak informasi yang dapat dimanfaatkan. Pengumpulan *tweet* berdasarkan radius lokasi, kemudian dikelompokkan berdasarkan topik tertentu, dan selanjutnya disusun menjadi sebuah *storytelling* dari kumpulan *tweet* yang memiliki topik yang sama akan menjadi bahan baru bagi para jurnalis.

# Questions for Investigation
1. Bagaimana kesesuaian hasil dalam bentuk *storytelling* pada topik yang sama?
2. Topik apakah yang paling sering dibicarakan di Twitter?

# Dataset
Dataset yang digunakan berupa 1000 *tweets* hasil *crawling* sendiri. Berikut adalah struktur dataset yang digunakan:
```
user         username pengguna twitter
created_at   tanggal dan waktu dibuatnya tweet
text         tweet
```
Dataset ditempatkan pada file dengan format *.csv*

# Crawling
#### Proses crawling
1. Menggunakan library dari python yaitu twitter, dilakukan crawling berdasarkan radius dari koordinat latitude dan longtitude
2. Data crawling meliputi atribut <b>user, created_at, text</b>
3. Hasil crawling disimpan pada file bertipe .csv

In [1]:
import tweepy
import time
import sys
import csv

latitude = -7.059035     # geographical centre of search
longitude = 110.443972   # geographical centre of search
max_range = 10            # search range in kilometres
outfile = "tweets2.csv"

consumer_key = 'w6LdCwZHkdrIaNgxvuhZziSYe'
consumer_secret = 'Y4guQcTeCwpXJ8UhYyYzCbazxTgHZqYLpN0maNJThXXCEfrMwz'
access_token = '477563933-CIawAP0XgC8tjjXcRYmKaf4p0w2OqFAG4duYCqsL'
access_secret = 'iZIjf2p4QaNyRE7wcibb1vNpIK2JntSRfrruoYRBMs6AS'

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [3]:
csvfile = open(outfile, "w", newline='', encoding='utf-8')
csvwriter = csv.writer(csvfile)

row = [ "username", "created_at", "text" ]
csvwriter.writerow(row)

26

In [4]:
result_count = 0
start_time = time.clock()

c = tweepy.Cursor(api.search,
                  q="*",
                  count=50000,
                  geocode = "%f,%f,%dkm" % (latitude, longitude, max_range)).items(50000)
while True:
    try:
        tweet = c.next()
        csvwriter.writerow([tweet.user.screen_name, tweet.created_at, tweet.text])
        result_count += 1
        print ("got %d results" % result_count)
    except tweepy.TweepError:
        time.sleep(15 * 60)
        continue
    except StopIteration:
        break
        
print(time.clock() - start_time, "seconds")

got 1 results
got 2 results
got 3 results
got 4 results
got 5 results
got 6 results
got 7 results
got 8 results
got 9 results
got 10 results
got 11 results
got 12 results
got 13 results
got 14 results
got 15 results
got 16 results
got 17 results
got 18 results
got 19 results
got 20 results
got 21 results
got 22 results
got 23 results
got 24 results
got 25 results
got 26 results
got 27 results
got 28 results
got 29 results
got 30 results
got 31 results
got 32 results
got 33 results
got 34 results
got 35 results
got 36 results
got 37 results
got 38 results
got 39 results
got 40 results
got 41 results
got 42 results
got 43 results
got 44 results
got 45 results
got 46 results
got 47 results
got 48 results
got 49 results
got 50 results
got 51 results
got 52 results
got 53 results
got 54 results
got 55 results
got 56 results
got 57 results
got 58 results
got 59 results
got 60 results
got 61 results
got 62 results
got 63 results
got 64 results
got 65 results
got 66 results
got 67 results
got 

got 595 results
got 596 results
got 597 results
got 598 results
got 599 results
got 600 results
got 601 results
got 602 results
got 603 results
got 604 results
got 605 results
got 606 results
got 607 results
got 608 results
got 609 results
got 610 results
got 611 results
got 612 results
got 613 results
got 614 results
got 615 results
got 616 results
got 617 results
got 618 results
got 619 results
got 620 results
got 621 results
got 622 results
got 623 results
got 624 results
got 625 results
got 626 results
got 627 results
got 628 results
got 629 results
got 630 results
got 631 results
got 632 results
got 633 results
got 634 results
got 635 results
got 636 results
got 637 results
got 638 results
got 639 results
got 640 results
got 641 results
got 642 results
got 643 results
got 644 results
got 645 results
got 646 results
got 647 results
got 648 results
got 649 results
got 650 results
got 651 results
got 652 results
got 653 results
got 654 results
got 655 results
got 656 results
got 657 

got 1195 results
got 1196 results
got 1197 results
got 1198 results
got 1199 results
got 1200 results
got 1201 results
got 1202 results
got 1203 results
got 1204 results
got 1205 results
got 1206 results
got 1207 results
got 1208 results
got 1209 results
got 1210 results
got 1211 results
got 1212 results
got 1213 results
got 1214 results
got 1215 results
got 1216 results
got 1217 results
got 1218 results
got 1219 results
got 1220 results
got 1221 results
got 1222 results
got 1223 results
got 1224 results
got 1225 results
got 1226 results
got 1227 results
got 1228 results
got 1229 results
got 1230 results
got 1231 results
got 1232 results
got 1233 results
got 1234 results
got 1235 results
got 1236 results
got 1237 results
got 1238 results
got 1239 results
got 1240 results
got 1241 results
got 1242 results
got 1243 results
got 1244 results
got 1245 results
got 1246 results
got 1247 results
got 1248 results
got 1249 results
got 1250 results
got 1251 results
got 1252 results
got 1253 resul

got 1690 results
got 1691 results
got 1692 results
got 1693 results
got 1694 results
got 1695 results
got 1696 results
got 1697 results
got 1698 results
got 1699 results
got 1700 results
got 1701 results
got 1702 results
got 1703 results
got 1704 results
got 1705 results
got 1706 results
got 1707 results
got 1708 results
got 1709 results
got 1710 results
got 1711 results
got 1712 results
got 1713 results
got 1714 results
got 1715 results
got 1716 results
got 1717 results
got 1718 results
got 1719 results
got 1720 results
got 1721 results
got 1722 results
got 1723 results
got 1724 results
got 1725 results
got 1726 results
got 1727 results
got 1728 results
got 1729 results
got 1730 results
got 1731 results
got 1732 results
got 1733 results
got 1734 results
got 1735 results
got 1736 results
got 1737 results
got 1738 results
got 1739 results
got 1740 results
got 1741 results
got 1742 results
got 1743 results
got 1744 results
got 1745 results
got 1746 results
got 1747 results
got 1748 resul

got 2185 results
got 2186 results
got 2187 results
got 2188 results
got 2189 results
got 2190 results
got 2191 results
got 2192 results
got 2193 results
got 2194 results
got 2195 results
got 2196 results
got 2197 results
got 2198 results
got 2199 results
got 2200 results
got 2201 results
got 2202 results
got 2203 results
got 2204 results
got 2205 results
got 2206 results
got 2207 results
got 2208 results
got 2209 results
got 2210 results
got 2211 results
got 2212 results
got 2213 results
got 2214 results
got 2215 results
got 2216 results
got 2217 results
got 2218 results
got 2219 results
got 2220 results
got 2221 results
got 2222 results
got 2223 results
got 2224 results
got 2225 results
got 2226 results
got 2227 results
got 2228 results
got 2229 results
got 2230 results
got 2231 results
got 2232 results
got 2233 results
got 2234 results
got 2235 results
got 2236 results
got 2237 results
got 2238 results
got 2239 results
got 2240 results
got 2241 results
got 2242 results
got 2243 resul

...

got 47512 results
got 47513 results
got 47514 results
got 47515 results
got 47516 results
got 47517 results
got 47518 results
got 47519 results
got 47520 results
got 47521 results
got 47522 results
got 47523 results
got 47524 results
got 47525 results
got 47526 results
got 47527 results
got 47528 results
got 47529 results
got 47530 results
got 47531 results
got 47532 results
got 47533 results
got 47534 results
got 47535 results
got 47536 results
got 47537 results
got 47538 results
got 47539 results
got 47540 results
got 47541 results
got 47542 results
got 47543 results
got 47544 results
got 47545 results
got 47546 results
got 47547 results
got 47548 results
got 47549 results
got 47550 results
got 47551 results
got 47552 results
got 47553 results
got 47554 results
got 47555 results
got 47556 results
got 47557 results
got 47558 results
got 47559 results
got 47560 results
got 47561 results
got 47562 results
got 47563 results
got 47564 results
got 47565 results
got 47566 results
got 47567 

got 48012 results
got 48013 results
got 48014 results
got 48015 results
got 48016 results
got 48017 results
got 48018 results
got 48019 results
got 48020 results
got 48021 results
got 48022 results
got 48023 results
got 48024 results
got 48025 results
got 48026 results
got 48027 results
got 48028 results
got 48029 results
got 48030 results
got 48031 results
got 48032 results
got 48033 results
got 48034 results
got 48035 results
got 48036 results
got 48037 results
got 48038 results
got 48039 results
got 48040 results
got 48041 results
got 48042 results
got 48043 results
got 48044 results
got 48045 results
got 48046 results
got 48047 results
got 48048 results
got 48049 results
got 48050 results
got 48051 results
got 48052 results
got 48053 results
got 48054 results
got 48055 results
got 48056 results
got 48057 results
got 48058 results
got 48059 results
got 48060 results
got 48061 results
got 48062 results
got 48063 results
got 48064 results
got 48065 results
got 48066 results
got 48067 

got 48512 results
got 48513 results
got 48514 results
got 48515 results
got 48516 results
got 48517 results
got 48518 results
got 48519 results
got 48520 results
got 48521 results
got 48522 results
got 48523 results
got 48524 results
got 48525 results
got 48526 results
got 48527 results
got 48528 results
got 48529 results
got 48530 results
got 48531 results
got 48532 results
got 48533 results
got 48534 results
got 48535 results
got 48536 results
got 48537 results
got 48538 results
got 48539 results
got 48540 results
got 48541 results
got 48542 results
got 48543 results
got 48544 results
got 48545 results
got 48546 results
got 48547 results
got 48548 results
got 48549 results
got 48550 results
got 48551 results
got 48552 results
got 48553 results
got 48554 results
got 48555 results
got 48556 results
got 48557 results
got 48558 results
got 48559 results
got 48560 results
got 48561 results
got 48562 results
got 48563 results
got 48564 results
got 48565 results
got 48566 results
got 48567 

got 49012 results
got 49013 results
got 49014 results
got 49015 results
got 49016 results
got 49017 results
got 49018 results
got 49019 results
got 49020 results
got 49021 results
got 49022 results
got 49023 results
got 49024 results
got 49025 results
got 49026 results
got 49027 results
got 49028 results
got 49029 results
got 49030 results
got 49031 results
got 49032 results
got 49033 results
got 49034 results
got 49035 results
got 49036 results
got 49037 results
got 49038 results
got 49039 results
got 49040 results
got 49041 results
got 49042 results
got 49043 results
got 49044 results
got 49045 results
got 49046 results
got 49047 results
got 49048 results
got 49049 results
got 49050 results
got 49051 results
got 49052 results
got 49053 results
got 49054 results
got 49055 results
got 49056 results
got 49057 results
got 49058 results
got 49059 results
got 49060 results
got 49061 results
got 49062 results
got 49063 results
got 49064 results
got 49065 results
got 49066 results
got 49067 

got 49512 results
got 49513 results
got 49514 results
got 49515 results
got 49516 results
got 49517 results
got 49518 results
got 49519 results
got 49520 results
got 49521 results
got 49522 results
got 49523 results
got 49524 results
got 49525 results
got 49526 results
got 49527 results
got 49528 results
got 49529 results
got 49530 results
got 49531 results
got 49532 results
got 49533 results
got 49534 results
got 49535 results
got 49536 results
got 49537 results
got 49538 results
got 49539 results
got 49540 results
got 49541 results
got 49542 results
got 49543 results
got 49544 results
got 49545 results
got 49546 results
got 49547 results
got 49548 results
got 49549 results
got 49550 results
got 49551 results
got 49552 results
got 49553 results
got 49554 results
got 49555 results
got 49556 results
got 49557 results
got 49558 results
got 49559 results
got 49560 results
got 49561 results
got 49562 results
got 49563 results
got 49564 results
got 49565 results
got 49566 results
got 49567 

In [5]:
import pandas as pd

In [191]:
df = pd.read_csv('tweets2.csv')
df.head()

,username,created_at,text
0,DqB6wxR6KDtzcYG,2017-10-23 04:43:53,RT @MamahPitri69: Tante lagi 😍😘\n=&gt; https:/...
1,Lilianamega26,2017-10-23 04:43:50,pak @ganjarpranowo tolong di colek dong bupati...
2,Iwan_ndok,2017-10-23 04:43:37,Baru saja mengirim foto https://t.co/ztLJAHgwqP
3,HPB_10,2017-10-23 04:43:23,"Sesaat aku menjadi embuh, kayak udah mau hari ..."
4,_FinesseDoll,2017-10-23 04:43:07,RT @_FinesseDoll: I'm such a cry baby😭I'm toug...


In [192]:
texts = df['text']
texts.head()

0    RT @MamahPitri69: Tante lagi 😍😘\n=&gt; https:/...
1    pak @ganjarpranowo tolong di colek dong bupati...
2      Baru saja mengirim foto https://t.co/ztLJAHgwqP
3    Sesaat aku menjadi embuh, kayak udah mau hari ...
4    RT @_FinesseDoll: I'm such a cry baby😭I'm toug...
Name: text, dtype: object

In [193]:
print(df['text'][4])
type(df['text'][4])

RT @_FinesseDoll: I'm such a cry baby😭I'm tough aslllllll but all you gotta do is hurt my feelings imma be crying 😂🤷🏽‍♀️


str

In [194]:
def encode_ascii_replace(row):
    return row.encode('ascii', 'replace')

In [195]:
t = df['text']
t = t.apply(encode_ascii_replace)

In [196]:
t[4].decode('utf-8')

"RT @_FinesseDoll: I'm such a cry baby?I'm tough aslllllll but all you gotta do is hurt my feelings imma be crying ??????"

In [197]:
def decode(row):
    return row.decode('utf-8')

In [198]:
td = t.apply(decode)

In [199]:
td.to_csv('text.csv', header=['tweets'], index=False)

## Text Data Cleaning

In [200]:
texts_df = pd.read_csv('text.csv')
texts_df.head()

,tweets
0,RT @MamahPitri69: Tante lagi ??\n=&gt; https:/...
1,pak @ganjarpranowo tolong di colek dong bupati...
2,Baru saja mengirim foto https://t.co/ztLJAHgwqP
3,"Sesaat aku menjadi embuh, kayak udah mau hari ..."
4,RT @_FinesseDoll: I'm such a cry baby?I'm toug...


In [201]:
import html

def html_parser(row):
    return html.unescape(row)

In [202]:
tdf = texts_df['tweets']
clean1 = tdf.apply(html_parser)
clean1[0]

'RT @MamahPitri69: Tante lagi ??\n=> https://t.co/cCf8ATsVDB https://t.co/Ckeq9BoD3j'

In [203]:
def decoding_data(row):
    return row.encode('ascii', 'ignore').decode('utf-8')

In [204]:
clean2 = clean1.apply(decoding_data)
clean2[53]

'RT @BEMUndip_: [SERUAN AKSI SMS SERENTAK]\n\nMasih adakah keadilan di Negeri ini? Ketika suara rakyat dibungkam\n\nRead: https://t.co/WNyu0JfQs?'

In [205]:
import itertools

def standardizing_words(row):
    return ''.join(''.join(s)[:2] for _, s in itertools.groupby(row) )

In [206]:
clean3 = clean2.apply(standardizing_words)
print(clean2[49])
clean3[49]

Teeeh? https://t.co/1pAlg0X2wm


'Teeh? https://t.co/1pAlg0X2wm'

In [207]:
import re

def clean_url(row):
    return re.sub(r"http\S+", "", row)

In [208]:
clean4 = clean3.apply(clean_url)
print(clean3[0])
clean4[0]

RT @MamahPitri69: Tante lagi ??
=> https://t.co/cCf8ATsVDB https://t.co/Ckeq9BoD3j


'RT @MamahPitri69: Tante lagi ??\n=>  '

In [ ]:
def remove_newline(row):
    return ' '.join(row.splitlines())

In [ ]:
clean5 = clean4.apply(remove_newline)
clean5[0]

In [ ]:
from string import maketrans

intab = "=><;()^{}[]\~"
outtab = ""
trantab = maketrans(intab, outtab)

def remove_unnecessary_punctuation(row):
    return row.translate(trantab)